# Tweet Downloader

**CS109a**: Fall 2018

**Authors**: Gordon Hew, Wenqin Hu, Blair Leduc

**TF**: Ken Arnold

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
import sys
import jsonpickle
import os
import zipfile
import time
from datetime import date, datetime, time
import re
import json
from pprint import pprint
import random
from dateutil import tz
import gzip
import shutil

# Parameters

In [3]:
enable_user_id_validation = False

In [4]:
# Constants, should not change
CONSUMER_KEY = 'qAm9baDI3rzzK2aUhQW3mzvNp'
CONSUMER_SECRET = "x4h2XK4uoCKobjBSuwjUIGc9AB2iOWvd5StzgSfTWloYrQBgEw"

ACCESS_TOKEN = '1368388352-wCcxn8BXKP4KsdQK40ztCMjCbG5Nhtm9FTXuw2c'
ACCESS_TOKEN_SECRET = 'oe3ptX7WcCbpIRH4ue8pyOjN8fpmm1k8O72OxwCc8igdx'

In [5]:
def get_twitter_api(consumer_key, consumer_secret, access_token, 
                    access_token_secret):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

    api = tweepy.API(auth, 
                     wait_on_rate_limit = True, 
                     wait_on_rate_limit_notify = True, 
                     retry_count = 10, 
                     retry_delay = 10, 
                     retry_errors = set([401, 404, 500, 503]))
    
    auth.set_access_token(access_token, access_token_secret) 

    if (not api):
        print ("Can't Authenticate")
        sys.exit(-1)
  
    return api

def isValidUser(api, user_id):
    try:
        user = api.get_user(user_id)
        is_protected = user._json['protected']

        if not is_protected:
            return True
        else:
            print('not including user id', user_id, 
                  'because tweets are protected')
            return False
    except Exception as err:
        print('Unable to process user id', user_id, err)
    
    return False

In [6]:
api = get_twitter_api(CONSUMER_KEY, CONSUMER_SECRET, 
                      ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [7]:
class AccountLinkedWebsiteStreamListener(tweepy.StreamListener):
    def __init__(self, max_count = 100):
        self.max_count = max_count
        self.count = 0
        self.saveFile = open('accounts.json', 'w')
        self.user_ids = set()
        super(AccountLinkedWebsiteStreamListener, self).__init__()

    def on_data(self, data):
        if (self.count < self.max_count):
            tweet_json = json.loads(data)
            
            if 'delete' in tweet_json:
                return True
            
            if tweet_json['lang'] != 'en':
                return True       
            
            if tweet_json['user']['lang'] != 'en':
                return True
            
            if tweet_json['user']['id'] in self.user_ids:
                return True
            
            self.saveFile.write(json.dumps(tweet_json['user']))
            self.saveFile.write('\n')
            self.user_ids.add(tweet_json['user']['id'])
            self.count = self.count + 1
            return True
        else:
            self.saveFile.close()
            return False    
        
    def on_error(self, status_code):
        print('Error occurred, status_code=', status_code)
        return False

In [6]:
streamListener = AccountLinkedWebsiteStreamListener(max_count = 10000)
accountStream = tweepy.Stream(auth=api.auth, listener=streamListener)
accountStream.sample()

In [8]:
# read json file, filter by english language, extract user names
account_data = []
with open('accounts.json') as f:
    for line in f:
        if len(line.strip()) != 0:
            account_data.append(json.loads(line))

sample_user_ids = set()

for account in account_data: 
    try:
        if isValidUser(api,account['id']) and enable_user_id_validation:
            sample_user_ids.add(account['id'])
    except:
        print('Error extracting sample user id from tweet')
        pprint(account)
        raise 

print(len(sample_user_ids))

Unable to process user id 842567245637337088 [{'code': 63, 'message': 'User has been suspended.'}]
not including user id 3298733402 because tweets are protected
not including user id 108735476 because tweets are protected
Unable to process user id 1070715576287252480 [{'code': 63, 'message': 'User has been suspended.'}]


Rate limit reached. Sleeping for: 195


not including user id 1025702863492661251 because tweets are protected
not including user id 344960349 because tweets are protected
not including user id 357120762 because tweets are protected
not including user id 949003320718344192 because tweets are protected
not including user id 191421000 because tweets are protected


Rate limit reached. Sleeping for: 661


Unable to process user id 1070787393756643328 [{'code': 63, 'message': 'User has been suspended.'}]
not including user id 1020871961692557312 because tweets are protected


...

Unable to process user id 894717797359550464 [{'code': 63, 'message': 'User has been suspended.'}]
not including user id 955159533323784193 because tweets are protected
not including user id 1038924171206647810 because tweets are protected
not including user id 321061912 because tweets are protected
not including user id 757814618345529344 because tweets are protected
Unable to process user id 796379964895621120 [{'code': 63, 'message': 'User has been suspended.'}]


Rate limit reached. Sleeping for: 472


not including user id 1023843105101963264 because tweets are protected
not including user id 1010396030859218946 because tweets are protected
Unable to process user id 905085990624100352 [{'code': 63, 'message': 'User has been suspended.'}]
Unable to process user id 905557122083106816 [{'code': 63, 'message': 'User has been suspended.'}]
not including user id 1060431062428409857 because tweets are protected


Rate limit reached. Sleeping for: 486


not including user id 3646736794 because tweets are protected
9954


In [12]:
random.seed(a = 42)
sample_user_ids = list(sample_user_ids)
sample_indices = random.sample(range(len(sample_user_ids)), 1000)
random_sample_user_ids = [sample_user_ids[i] for i in sample_indices]

with open('random_user_ids.txt', 'w') as f:
    for user_id in random_sample_user_ids:
        f.write("%s\n" % user_id)

In [6]:
random_sample_user_ids = list()
with open('random_user_ids.txt', 'r') as f:
    for line in f:
        random_sample_user_ids.append(int(line))    

len(random_sample_user_ids)

1000

In [7]:
# user_index = pd.DataFrame()
with open('last_100_timeline_tweets.json', 'w') as f:

    for idx, random_sample_user_id in enumerate(random_sample_user_ids):

        print("Processing {}/{} user id {} ".format(idx + 1, 
                                        len(random_sample_user_ids), 
                                        random_sample_user_id))

        # we want to include the user_info for the latest tweet in this 
        # set which is a snapshot of the statuses, followers, etc. at 
        # thatpoint in time
        include_user_info = True

        try:
        
            # cursor gets tweets in desc timestamp order
            for status in tweepy.Cursor(api.user_timeline, 
                                        id = random_sample_user_id)\
                                                .items(100):

                if not include_user_info:
                    status._json['user_id'] = status._json['user']['id']
                    status._json.pop('user', None)
                else:
                    include_user_info = False

                f.write(json.dumps(status._json))
                f.write('\n')
        except Exception as err:
            print('Unable to process user id', 
                  random_sample_user_id, 
                  err)

Processing 1/1000 user id 934576158305345536 


Rate limit reached. Sleeping for: 202


Processing 2/1000 user id 3133965632 
Processing 3/1000 user id 3167730160 
Processing 4/1000 user id 893957155 
Processing 5/1000 user id 540553113 
Processing 6/1000 user id 1016110398 
Processing 7/1000 user id 83694940 
Processing 8/1000 user id 636744426 
Processing 9/1000 user id 279810488 
Processing 10/1000 user id 1957657604 
Processing 11/1000 user id 548493684 
Processing 12/1000 user id 788646721072922624 
Processing 13/1000 user id 3308193378 
Processing 14/1000 user id 1065127340118687744 
Processing 15/1000 user id 429502093 
Processing 16/1000 user id 16462191 
Processing 17/1000 user id 465005264 
Processing 18/1000 user id 1058549422936727552 
Processing 19/1000 user id 872698498688352256 
Processing 20/1000 user id 24344111 
Processing 21/1000 user id 2436409854 
Processing 22/1000 user id 3096834772 
Processing 23/1000 user id 2553174240 
Processing 24/1000 user id 2482646808 
Processing 25/1000 user id 350740243 
Processing 26/1000 user id 789403126654603264 
Proce

Rate limit reached. Sleeping for: 712


Processing 177/1000 user id 957253999580893186 
Processing 178/1000 user id 2867741109 
Processing 179/1000 user id 851241653692239873 
Processing 180/1000 user id 1028118993306087424 
Processing 181/1000 user id 4926037201 
Processing 182/1000 user id 846410393698238464 
Processing 183/1000 user id 176545684 
Processing 184/1000 user id 2325964293 
Processing 185/1000 user id 791440854292443136 
Processing 186/1000 user id 775629882671804416 
Processing 187/1000 user id 4174646658 
Processing 188/1000 user id 4910630839 
Processing 189/1000 user id 923486071282896898 
Processing 190/1000 user id 3096574850 
Processing 191/1000 user id 2241262770 
Processing 192/1000 user id 941225069149413376 
Processing 193/1000 user id 50920331 
Processing 194/1000 user id 3033575187 
Processing 195/1000 user id 340918587 
Processing 196/1000 user id 1009500239491883009 
Processing 197/1000 user id 845355200638586880 
Processing 198/1000 user id 2300162222 
Processing 199/1000 user id 70749792 
Proc

...

Processing 525/1000 user id 2171628228 
Processing 526/1000 user id 788189963351367680 
Processing 527/1000 user id 37548901 
Processing 528/1000 user id 1123802702 
Processing 529/1000 user id 1026560937032146944 
Processing 530/1000 user id 831692637358940160 
Processing 531/1000 user id 3308248890 
Processing 532/1000 user id 3597579975 
Processing 533/1000 user id 2388341916 
Processing 534/1000 user id 48609897 
Processing 535/1000 user id 896915042884501506 
Processing 536/1000 user id 407987861 
Processing 537/1000 user id 617919229 
Processing 538/1000 user id 1119467994 
Processing 539/1000 user id 3002850576 
Processing 540/1000 user id 957018576917073928 
Processing 541/1000 user id 144784400 
Processing 542/1000 user id 1025554838090907648 
Processing 543/1000 user id 740089702074580992 
Processing 544/1000 user id 624412502 
Processing 545/1000 user id 1187365764 
Processing 546/1000 user id 3196416447 
Processing 547/1000 user id 1084925322 
Processing 548/1000 user id 17

Rate limit reached. Sleeping for: 705


Processing 699/1000 user id 2559636770 
Processing 700/1000 user id 3440181735 
Processing 701/1000 user id 12512542 
Processing 702/1000 user id 1055270161744199680 
Processing 703/1000 user id 786794250176311298 
Processing 704/1000 user id 790672226 
Processing 705/1000 user id 1679914232 
Processing 706/1000 user id 1064086113252327424 
Processing 707/1000 user id 954260301624455169 
Processing 708/1000 user id 1909581740 
Processing 709/1000 user id 524836548 
Processing 710/1000 user id 2579989416 
Processing 711/1000 user id 3323433771 
Processing 712/1000 user id 3193847263 
Processing 713/1000 user id 3835408217 
Processing 714/1000 user id 793040312 
Processing 715/1000 user id 749742651000389632 
Processing 716/1000 user id 4831199386 
Processing 717/1000 user id 3300751913 
Processing 718/1000 user id 1021157181830451202 
Processing 719/1000 user id 4730086461 
Processing 720/1000 user id 156267662 
Processing 721/1000 user id 74838026 
Processing 722/1000 user id 757831213

Rate limit reached. Sleeping for: 724


Processing 874/1000 user id 846836569721880576 
Processing 875/1000 user id 3111119063 
Processing 876/1000 user id 3291102316 
Processing 877/1000 user id 991656256019443713 
Processing 878/1000 user id 22574081 
Processing 879/1000 user id 3249637339 
Processing 880/1000 user id 193814597 
Processing 881/1000 user id 1337922031 
Processing 882/1000 user id 787333740 
Processing 883/1000 user id 1033710254775652352 
Processing 884/1000 user id 70436015 
Processing 885/1000 user id 2526353322 
Processing 886/1000 user id 1038374316 
Processing 887/1000 user id 872867308149944320 
Processing 888/1000 user id 2813755406 
Processing 889/1000 user id 1022920679061180416 
Processing 890/1000 user id 389636513 
Processing 891/1000 user id 1069206032046419968 
Processing 892/1000 user id 81365927 
Processing 893/1000 user id 3288606505 
Processing 894/1000 user id 24800640 
Processing 895/1000 user id 850608419094503424 
Processing 896/1000 user id 1043851674 
Processing 897/1000 user id 3980

In [9]:
line_num = 0
with open('last_100_timeline_tweets.json', 'r') as f:
    for line in f:
        
        if line_num == 0:
            pprint(json.loads(line))
            break
        
        line_num = line_num + 1

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Dec 08 01:50:37 +0000 2018',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [],
              'user_mentions': [{'id': 232901331,
                                 'id_str': '232901331',
                                 'indices': [0, 9],
                                 'name': 'Dan Bongino',
                                 'screen_name': 'dbongino'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 1071220471134347264,
 'id_str': '1071220471134347264',
 'in_reply_to_screen_name': 'dbongino',
 'in_reply_to_status_id': 1071195698102722560,
 'in_reply_to_status_id_str': '1071195698102722560',
 'in_reply_to_user_id': 232901331,
 'in_reply_to_user_id_str': '232901331',
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter '
           'Lite</a>',
 'text': '@d

In [2]:
with open('last_100_timeline_tweets.json', 'rb') as f_in:
    with gzip.open('last_100_timeline_tweets.json.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

FileNotFoundError: [Errno 2] No such file or directory: 'last_100_timeline_tweets.json'